In [1]:
!curl -0 https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt > input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1089k  100 1089k    0     0  12.3M      0 --:--:-- --:--:-- --:--:-- 12.8M


In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in chars", len(text))

length of dataset in chars 1115394


In [4]:
alphabet = sorted(list(set(text)))
vocab_size = len(alphabet)
print(''.join(alphabet))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = { ch:i for i, ch in enumerate(alphabet) }
itos = { i:ch for i, ch in enumerate(alphabet) }
encode = lambda s: [stoi[c] for c in s] # encodes string as a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # tae a list of integers, returns a string

print(encode("Hello World"))
print(decode(encode("Hello World")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [8]:
!pip install torch

     ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
     ---------------------------------------- 0.2/198.6 MB 7.6 MB/s eta 0:00:27
     --------------------------------------- 1.5/198.6 MB 18.9 MB/s eta 0:00:11
      -------------------------------------- 3.3/198.6 MB 26.5 MB/s eta 0:00:08
     - ------------------------------------- 5.3/198.6 MB 30.7 MB/s eta 0:00:07
     - ------------------------------------- 7.2/198.6 MB 33.1 MB/s eta 0:00:06
     - ------------------------------------- 9.0/198.6 MB 34.0 MB/s eta 0:00:06
     -- ----------------------------------- 10.7/198.6 MB 38.6 MB/s eta 0:00:05
     -- ----------------------------------- 12.8/198.6 MB 40.9 MB/s eta 0:00:05
     -- ----------------------------------- 13.7/198.6 MB 43.5 MB/s eta 0:00:05
     --- ---------------------------------- 16.6/198.6 MB 40.9 MB/s eta 0:00:05
     --- ---------------------------------- 18.5/198.6 MB 40.9 MB/s eta 0:00:05
     --- ---------------------------------- 20.

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\chris\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\torch\\ao\\pruning\\_experimental\\pruner\\saliency_pruner.py'
Check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\chris\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached torch-2.2.2-cp310-cp310-win_amd64.whl (198.6 MB)
  Using cached filelock-3.13.4-py3-none-any.whl (11 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached MarkupSafe-2.1.5-cp310-cp310-win_amd64.whl (17 kB)



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\chris\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
# Train test split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[:n]

In [12]:
torch.manual_seed(1337)
batch_size = 4 # number of independent sequences processed in parallel
block_size = 8 # max context length for predictions

# generate a small batch of data with inputs x and targets y
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


xb, yb = get_batch('train')

print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [16]:
import torch.nn as nn 
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch, Time, Channel)
        
        if targets == None:
            loss = None
        else:
            # Reshape
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            
            logits = logits[:, -1, :] # becomes (B, C)
            
            probs = F.softmax(logits, dim=-1)
            
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            
        return idx
    
m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.5262, grad_fn=<NllLossBackward0>)

'JgC.JZWqUkpdtkSpmzjM-,RqzgaN?vC:hgjnAnBZDga-APqGUH!WdCbIb;$DefOYbEvcaKGMmnO'q$KdS-'ZH
.YSqr'X!Q! d;


In [17]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [21]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    
print(loss.item())

2.5347774028778076


In [24]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



DUNju oveierk,
The s; witwhealengr ho ry
The e t
P re rsts wa lk l, d bl, cot thengl attizXNGBENTorooofusis stl a ant! brd py wnd:

Whe sen aveio:
DUShipizand, s dr wisow ghisepougldind
Fintu ctalad ppaild ile a a-
AGo ly me outermy:
O, itllped wir.
Bupooun t f enonshal icerdiggay harenwo! xcr's.

Tor f w IUNESTonured f RDelveeredis thakis thallditthave inde'd Hait whe d m!
NCHAPres. t beredes bju dgrst h ters, nshow hed hicorine sitheean nes lees PRDulefre man we, n the IOMNalflivammpavere cen
